# Pure GP Model Experiment

**CRITICAL INSIGHT**: The CV-LB gap is STRUCTURAL, not due to overfitting.
- exp_041 (aggressive regularization): CV 0.0090, LB 0.0932, ratio 10.36x - ON THE SAME LINE
- CV-LB relationship: LB = 4.22 × CV + 0.0534 (R² = 0.955)
- Intercept (0.0534) > target (0.0347) - current approach CANNOT reach target

**Hypothesis**: GP has fundamentally different mathematical framework. It may have a different CV-LB relationship (lower intercept).

**Why GP?**
1. GPs are Bayesian - they provide uncertainty estimates
2. GPs have built-in regularization through the kernel
3. GPs may extrapolate differently to unseen solvents
4. exp_030 showed GP helps (2.4% CV improvement, 1.1% LB improvement)

**Implementation**:
- Pure GP model (no MLP, no LGBM)
- Use Matern kernel with optimized hyperparameters
- Train one GP per target (3 GPs total)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel, ConstantKernel, RBF
from sklearn.preprocessing import StandardScaler
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

print('Imports complete')

Imports complete


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = ~((X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"]))
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups - use Spange only for GP (simpler, more interpretable)
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
print(f'Spange: {SPANGE_DF.shape}')

Spange: (26, 13)


In [4]:
# Simple Featurizer for GP - 18 features (Spange + Arrhenius kinetics)
class GPFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1]  # 18 features

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange])

print(f'GP feature dimension: {GPFeaturizer().feats_dim}')

GP feature dimension: 18


In [5]:
# Pure GP Model
class PureGPModel:
    """Pure Gaussian Process model - no MLP, no LGBM.
    
    Uses Matern kernel with automatic hyperparameter optimization.
    Trains one GP per target (3 GPs total).
    """
    def __init__(self, data='single'):
        self.data_type = data
        self.featurizer = GPFeaturizer(mixed=(data == 'full'))
        self.scaler = StandardScaler()
        self.y_scaler = StandardScaler()
        self.gps = []  # One GP per target
        
    def train_model(self, X, Y):
        # Featurize
        X_feat = self.featurizer.featurize(X)
        Y_np = Y.values
        
        # Scale
        X_scaled = self.scaler.fit_transform(X_feat)
        Y_scaled = self.y_scaler.fit_transform(Y_np)
        
        # Train GPs (one per target)
        self.gps = []
        for i in range(3):
            # Use Matern kernel with automatic hyperparameter optimization
            kernel = ConstantKernel(1.0, (1e-3, 1e3)) * Matern(length_scale=1.0, length_scale_bounds=(1e-2, 1e2), nu=2.5) + WhiteKernel(noise_level=0.1, noise_level_bounds=(1e-5, 1e1))
            gp = GaussianProcessRegressor(
                kernel=kernel, 
                n_restarts_optimizer=5,  # More restarts for better optimization
                normalize_y=True, 
                random_state=42
            )
            gp.fit(X_scaled, Y_scaled[:, i])
            self.gps.append(gp)
    
    def predict(self, X):
        # Featurize
        X_feat = self.featurizer.featurize(X)
        
        # Scale
        X_scaled = self.scaler.transform(X_feat)
        
        # GP prediction
        gp_pred_scaled = np.column_stack([gp.predict(X_scaled) for gp in self.gps])
        
        # Inverse transform
        gp_pred = self.y_scaler.inverse_transform(gp_pred_scaled)
        
        # Clip to valid range
        gp_pred = np.clip(gp_pred, 0, 1)
        
        return torch.tensor(gp_pred)
    
    def predict_with_tta(self, X):
        """Predict with test-time augmentation for mixed solvents."""
        if self.data_type == 'single':
            return self.predict(X)
        
        # For mixed solvents, average predictions from both orderings
        pred1 = self.predict(X)
        
        # Create flipped version
        X_flip = X.copy()
        X_flip["SOLVENT A NAME"] = X["SOLVENT B NAME"]
        X_flip["SOLVENT B NAME"] = X["SOLVENT A NAME"]
        X_flip["SolventB%"] = 1 - X["SolventB%"]
        
        # Featurize flipped
        X_feat_flip = self.featurizer.featurize(X_flip, flip=False)
        X_scaled_flip = self.scaler.transform(X_feat_flip)
        
        gp_pred_scaled = np.column_stack([gp.predict(X_scaled_flip) for gp in self.gps])
        pred2 = self.y_scaler.inverse_transform(gp_pred_scaled)
        pred2 = np.clip(pred2, 0, 1)
        pred2 = torch.tensor(pred2)
        
        # Average
        return (pred1 + pred2) / 2

print('PureGPModel defined')

PureGPModel defined


In [6]:
# Test the model
X_single, Y_single = load_data("single_solvent")
print(f"Single solvent data: {X_single.shape[0]} samples, {len(X_single['SOLVENT NAME'].unique())} unique solvents")

X_full, Y_full = load_data("full")
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
print(f"Full data: {X_full.shape[0]} samples, {len(ramps)} unique ramps")

Single solvent data: 656 samples, 24 unique solvents
Full data: 1227 samples, 13 unique ramps


In [7]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

import tqdm

X, Y = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PureGPModel(data='single')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_single_solvent = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE THIRD LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [00:36, 36.86s/it]

2it [01:15, 37.70s/it]

3it [01:48, 35.91s/it]

4it [02:23, 35.29s/it]

...

21it [13:08, 38.09s/it]

22it [13:47, 38.42s/it]

23it [14:24, 38.00s/it]

24it [15:03, 38.26s/it]

24it [15:03, 37.63s/it]

In [8]:
########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

X, Y = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X, Y)
all_predictions = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator)):
    (train_X, train_Y), (test_X, test_Y) = split

    model = PureGPModel(data='full')  # CHANGE THIS LINE ONLY
    model.train_model(train_X, train_Y)

    predictions = model.predict_with_tta(test_X)  # Shape: [N, 3]

    # Move to CPU and convert to numpy
    predictions_np = predictions.detach().cpu().numpy()

    # Add metadata and flatten to long format
    for row_idx, row in enumerate(predictions_np):
        all_predictions.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })

# Save final submission
submission_full_data = pd.DataFrame(all_predictions)

########### DO NOT CHANGE ANYTHING IN THIS CELL OTHER THAN THE MODEL #################
########### THIS MUST BE THE SECOND LAST CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

0it [00:00, ?it/s]

1it [01:48, 108.38s/it]

2it [03:31, 105.22s/it]

3it [05:18, 105.94s/it]

4it [07:01, 104.94s/it]

...

10it [18:02, 110.22s/it]

11it [20:00, 112.75s/it]

12it [22:01, 115.17s/it]

13it [23:52, 113.97s/it]

13it [23:52, 110.20s/it]

In [9]:
########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

########### DO NOT CHANGE ANYTHING IN THIS CELL #################
########### THIS MUST BE THE FINAL CELL IN YOUR NOTEBOOK FOR A VALID SUBMISSION #################

In [10]:
# Calculate CV score (for verification only - NOT part of submission)
X_single, Y_single = load_data("single_solvent")
X_full, Y_full = load_data("full")

# Get actuals in same order as predictions
actuals_single = []
for solvent in sorted(X_single["SOLVENT NAME"].unique()):
    mask = X_single["SOLVENT NAME"] == solvent
    actuals_single.append(Y_single[mask].values)
actuals_single = np.vstack(actuals_single)

actuals_full = []
ramps = X_full[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
for _, row in ramps.iterrows():
    mask = (X_full["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X_full["SOLVENT B NAME"] == row["SOLVENT B NAME"])
    actuals_full.append(Y_full[mask].values)
actuals_full = np.vstack(actuals_full)

# Get predictions
preds_single = submission_single_solvent[['target_1', 'target_2', 'target_3']].values
preds_full = submission_full_data[['target_1', 'target_2', 'target_3']].values

# Calculate MSE
mse_single = np.mean((actuals_single - preds_single) ** 2)
mse_full = np.mean((actuals_full - preds_full) ** 2)
n_single = len(actuals_single)
n_full = len(actuals_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== PURE GP MODEL CV SCORE ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')

print(f'\n=== COMPARISON ===')
print(f'exp_032 (best CV, GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194')
print(f'exp_044 (PURE GP): CV {overall_mse:.6f}')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than exp_032!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than exp_032')
    
# Estimate LB using old relationship
estimated_lb = 4.22 * overall_mse + 0.0534
print(f'\nEstimated LB (using old relationship): {estimated_lb:.4f}')
print(f'Best LB so far: 0.0877')
print(f'Target: 0.0347')
print(f'\nKey question: Does Pure GP have a DIFFERENT CV-LB relationship?')
print(f'If Pure GP has lower intercept, it might reach the target even with worse CV.')


=== PURE GP MODEL CV SCORE ===
Single Solvent MSE: 0.014034 (n=656)
Full Data MSE: 0.014754 (n=1227)
Overall MSE: 0.014503

=== COMPARISON ===
exp_032 (best CV, GP 0.15 + MLP 0.55 + LGBM 0.3): CV 0.008194
exp_044 (PURE GP): CV 0.014503

✗ WORSE: 77.00% worse than exp_032

Estimated LB (using old relationship): 0.1146
Best LB so far: 0.0877
Target: 0.0347

Key question: Does Pure GP have a DIFFERENT CV-LB relationship?
If Pure GP has lower intercept, it might reach the target even with worse CV.
